**Primera Iteración:** Configuración inicial de embeddings, sin balanceo de la data y sin sintonización del modelo.

In [ ]:
import torch
!pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

torch.manual_seed(11)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#leer el archivo fraudTrain.csv
df = pd.read_csv('fraudTrain.csv')

In [ ]:
df.shape

(1296675, 23)

In [ ]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
from torch_geometric.nn import Node2Vec

# Crear un grafo bipartito en NetworkX
B = nx.Graph()

# Agregar nodos de tipo "cc_num" y "merchant"
for index, row in df.iterrows():
    B.add_node(row['cc_num'], bipartite=0)  # Nodos de tarjetas
    B.add_node(row['merchant'], bipartite=1)  # Nodos de comerciantes
    B.add_edge(row['cc_num'], row['merchant'], is_fraud=row['is_fraud'])

# Convertir el grafo NetworkX a un formato compatible con PyTorch Geometric
data = from_networkx(B)

In [ ]:
#caclular los nodos del grafo
num_nodes = data.num_nodes
num_nodes

1676

In [ ]:
#calcular las aristas
num_edges = data.num_edges
num_edges

958144

In [ ]:
model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                 context_size=10, walks_per_node=10,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

# Obtener los embeddings de los nodos
with torch.no_grad():
    model.eval()
    z = model()  # Calcular los embeddings de los nodos
# z contiene los embeddings de los nodos.

Epoch: 01, Loss: 9.2870
Epoch: 02, Loss: 7.8863
Epoch: 03, Loss: 6.7240
Epoch: 04, Loss: 5.7070
Epoch: 05, Loss: 4.8872
Epoch: 06, Loss: 4.1995
Epoch: 07, Loss: 3.6521
Epoch: 08, Loss: 3.1974
Epoch: 09, Loss: 2.8271
Epoch: 10, Loss: 2.5334
Epoch: 11, Loss: 2.2903
Epoch: 12, Loss: 2.1016
Epoch: 13, Loss: 1.9499
Epoch: 14, Loss: 1.8362
Epoch: 15, Loss: 1.7376
Epoch: 16, Loss: 1.6676
Epoch: 17, Loss: 1.6083
Epoch: 18, Loss: 1.5630
Epoch: 19, Loss: 1.5267
Epoch: 20, Loss: 1.4987
Epoch: 21, Loss: 1.4752
Epoch: 22, Loss: 1.4572
Epoch: 23, Loss: 1.4413
Epoch: 24, Loss: 1.4314
Epoch: 25, Loss: 1.4212
Epoch: 26, Loss: 1.4123
Epoch: 27, Loss: 1.4057
Epoch: 28, Loss: 1.3993
Epoch: 29, Loss: 1.3952
Epoch: 30, Loss: 1.3913
Epoch: 31, Loss: 1.3882
Epoch: 32, Loss: 1.3848
Epoch: 33, Loss: 1.3823
Epoch: 34, Loss: 1.3797
Epoch: 35, Loss: 1.3768
Epoch: 36, Loss: 1.3748
Epoch: 37, Loss: 1.3732
Epoch: 38, Loss: 1.3723
Epoch: 39, Loss: 1.3715
Epoch: 40, Loss: 1.3691
Epoch: 41, Loss: 1.3687
Epoch: 42, Loss:

In [ ]:
# Obtener los embeddings finales
z = model()
z

Parameter containing:
tensor([[ 0.0561, -0.0812,  0.1429,  ..., -0.0867,  0.0527,  0.0087],
        [ 0.0634, -0.0397, -0.0528,  ..., -0.0185, -0.1001,  0.0892],
        [ 0.1194, -0.0818,  0.0293,  ..., -0.0788,  0.0380,  0.1563],
        ...,
        [-0.2939, -0.0044,  0.1179,  ...,  0.4593,  0.4371,  0.0435],
        [-0.2264, -0.2873,  0.1062,  ...,  0.4270, -0.2817, -0.2225],
        [-0.1440,  0.0329, -0.1937,  ...,  0.2915, -0.0229,  0.1266]],
       device='cuda:0', requires_grad=True)

In [ ]:
len(z)

1676

In [ ]:
# Crear una lista para almacenar los embeddings combinados
combined_embeddings = []

# Iterar a través de las filas del DataFrame
for index, row in df.iterrows():
  # Obtener el ID del nodo origen (cc_num) y el ID del nodo destino (merchant)
  source_node_id = list(B.nodes).index(row['cc_num'])
  target_node_id = list(B.nodes).index(row['merchant'])

  # Obtener los embeddings del nodo origen y el nodo destino
  source_embedding = z[source_node_id].detach().cpu().numpy()
  target_embedding = z[target_node_id].detach().cpu().numpy()

  # Combinar los embeddings (puedes usar concatenación, suma, multiplicación, etc.)
  # Aquí se utiliza la concatenación como ejemplo
  combined_embedding = np.concatenate((source_embedding, target_embedding))
  combined_embeddings.append(combined_embedding)

# Agregar la lista de embeddings combinados al DataFrame
df['combined_embedding'] = combined_embeddings

In [ ]:
df['combined_embedding'].head()

,combined_embedding
0,"[0.056124587, -0.08118352, 0.1429416, 0.015325..."
1,"[0.119403414, -0.0818374, 0.029305413, 0.00941..."
2,"[-0.018725231, -0.08215407, -0.12144027, -0.03..."
3,"[-0.019089427, 0.005568335, -0.033740252, 0.01..."
4,"[-0.019769173, -0.03858141, -0.095123805, -0.0..."


In [ ]:
df.to_csv('df_exported.csv', index=False)

In [ ]:
# División de train (70), test (15) y validacion (15)

from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, prueba y validación
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_df))
print("Tamaño del conjunto de prueba:", len(test_df))
print("Tamaño del conjunto de validación:", len(val_df))

Tamaño del conjunto de entrenamiento: 907672
Tamaño del conjunto de prueba: 194502
Tamaño del conjunto de validación: 194501


In [ ]:
type(X_train)

pandas.core.series.Series

In [ ]:
type(y_train)

pandas.core.series.Series

In [ ]:
#Entrenar modelo random forest clasifier para identificar is_fraud =1 o 0, utilizando solo train_df

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Preparar los datos de entrenamiento
X_train = [embedding for embedding in train_df['combined_embedding']]
y_train = train_df['is_fraud']

# Crear y entrenar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
import pickle

# Guardar el modelo en un archivo .pkl
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

print("Modelo guardado en rf_model.pkl")

Modelo guardado en rf_model.pkl


In [ ]:
# Calcular metricas en train, test y val

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

# Preparar los datos de prueba
X_test = [embedding for embedding in test_df['combined_embedding']]
y_test = test_df['is_fraud']

# Preparar los datos de validación
X_val = [embedding for embedding in val_df['combined_embedding']]
y_val = val_df['is_fraud']


# Realizar predicciones en los conjuntos de datos
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)
y_val_pred = rf_model.predict(X_val)


# Calcular las métricas para los tres conjuntos de datos
def calculate_metrics(y_true, y_pred):
  f1 = f1_score(y_true, y_pred)
  accuracy = accuracy_score(y_true, y_pred)
  try:
      auc = roc_auc_score(y_true, y_pred)
  except ValueError:
      auc = 0  # O manejar el error de otra manera
  return f1, accuracy, auc

train_f1, train_accuracy, train_auc = calculate_metrics(y_train, y_train_pred)
test_f1, test_accuracy, test_auc = calculate_metrics(y_test, y_test_pred)
val_f1, val_accuracy, val_auc = calculate_metrics(y_val, y_val_pred)

print("Métricas para el conjunto de entrenamiento:")
print(f"  F1-score: {train_f1:.4f}")
print(f"  Accuracy: {train_accuracy:.4f}")
print(f"  AUC: {train_auc:.4f}")

print("\nMétricas para el conjunto de prueba:")
print(f"  F1-score: {test_f1:.4f}")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"  AUC: {test_auc:.4f}")

print("\nMétricas para el conjunto de validación:")
print(f"  F1-score: {val_f1:.4f}")
print(f"  Accuracy: {val_accuracy:.4f}")
print(f"  AUC: {val_auc:.4f}")

Métricas para el conjunto de entrenamiento:
  F1-score: 0.5455
  Accuracy: 0.9964
  AUC: 0.6903

Métricas para el conjunto de prueba:
  F1-score: 0.1355
  Accuracy: 0.9928
  AUC: 0.5455

Métricas para el conjunto de validación:
  F1-score: 0.1459
  Accuracy: 0.9934
  AUC: 0.5484
